# Neural Network: Encoding Categorical Variables on 'Arrest'

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import tensorflow as tf
import matplotlib as plt
import sklearn as skl
from datetime import datetime
from sklearn.metrics import confusion_matrix
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect

In [2]:
#  Import and read the Chicago_Crimes_cleaned.csv.
import pandas as pd 
cc_df = pd.read_csv("Resources/Chicago_Crimes_cleaned_pt2.csv")
cc_df.head()

,ID,Offense,Offense_Description,Arrest,Domestic,DateTime,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day,Place,Police_Dist,Community_No,Community_Name,Community_Group,Latitude,Longitude,LatLong
0,12283861,NON-VIOLENT OTHER,UNLAWFUL POSSESSION - HANDGUN,True,False,2021-02-02 00:57:00,1612227420000000000,2021-02-02,1,Tuesday,00:57:00,Night,STREET,3.0,69.0,Greater Grand Crossing,South,41.763110,-87.615232,"(41.763110324, -87.615231799)"
1,11767977,THEFT,$500 AND UNDER,False,False,2019-07-21 17:00:00,1563728400000000000,2019-07-21,6,Sunday,17:00:00,Afternoon,Other,3.0,69.0,Greater Grand Crossing,South,41.762927,-87.595774,"(41.762926686, -87.595774261)"
2,12308286,ASSAULT,SIMPLE,False,False,2021-03-06 02:45:00,1614998700000000000,2021-03-06,5,Saturday,02:45:00,Early Morning,RESIDENCE,3.0,69.0,Greater Grand Crossing,South,41.767020,-87.595181,"(41.767020109, -87.595181198)"
3,11681265,BATTERY,SIMPLE,False,False,2019-05-08 20:00:00,1557345600000000000,2019-05-08,2,Wednesday,20:00:00,Evening,RESIDENCE,7.0,69.0,Greater Grand Crossing,South,41.770208,-87.628291,"(41.770208372, -87.628290908)"
4,11831071,THEFT,$500 AND UNDER,False,False,2019-09-13 07:30:00,1568359800000000000,2019-09-13,4,Friday,07:30:00,Morning,STREET,7.0,69.0,Greater Grand Crossing,South,41.764603,-87.633422,"(41.764602959, -87.633421657)"


In [3]:
# Create an engine that can talk to the database
engine = create_engine('sqlite:///myDatabase.db', echo=False)

In [4]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
Base.classes.keys()

['Arrest_info',
 'Arrest_info1',
 'Arrest_information',
 'Arrest_location2',
 'DateTime_info',
 'Offense1',
 'Offense2',
 'Offense_info']

In [6]:
# Get the name of the table. 
inspector = inspect(engine)
inspector.get_table_names()

['Arrest_info',
 'Arrest_info1',
 'Arrest_information',
 'Arrest_location2',
 'Chicago_Crime_data',
 'Chicago_Crime_info',
 'DateTime_info',
 'Offense1',
 'Offense2',
 'Offense_info',
 'chicago_crime_data_info',
 'crime_data_info',
 'pre_encoded']

In [7]:
columns = inspector.get_columns('chicago_crime_data_info')
for column in columns:
    print(column)

{'name': 'ID', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'DateTime', 'type': NUMERIC(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Offense', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Place', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Arrest', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Domestic', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Latitude', 'type': REAL(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Longitude', 'type': REAL(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'LatLong', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement

In [8]:
chicago_crime_df = pd.read_sql_table('chicago_crime_data_info', 'sqlite:///myDatabase.db')#.drop(columns=['index'])
chicago_crime_df.head()

,ID,DateTime,Offense,Place,Arrest,Domestic,Latitude,Longitude,LatLong,Police_Dist,Community_No,Community_Name,Community_Group,Date,DateTime_Numeric,Day_Num,Day_SMTWTFS,Time,Time_of_Day
0,24397,2019-02-05 23:16:00,HOMICIDE,STREET,True,False,41.751181,-87.663461,"(41.751181266, -87.663460692)",6.0,71.0,Auburn Gresham,Far Soutwest,2019-02-05,1549408560000000000,1,Tuesday,23:16:00,Night
1,24426,2019-03-08 01:04:00,HOMICIDE,ALLEY,True,False,41.893646,-87.635629,"(41.893646347, -87.635628657)",18.0,8.0,Near North Side,Central,2019-03-08,1552007040000000000,4,Friday,01:04:00,Night
2,24430,2019-03-13 00:50:00,HOMICIDE,STREET,False,False,41.750322,-87.666211,"(41.750321833, -87.666210811)",6.0,71.0,Auburn Gresham,Far Soutwest,2019-03-13,1552438200000000000,2,Wednesday,00:50:00,Night
3,24437,2019-03-17 13:54:00,HOMICIDE,Other,False,False,41.793773,-87.689421,"(41.793772993, -87.689420934)",9.0,63.0,Gage Park,Southwest,2019-03-17,1552830840000000000,6,Sunday,13:54:00,Midday
4,24451,2019-03-28 00:55:00,HOMICIDE,Other,False,False,41.740262,-87.557553,"(41.740262274, -87.557553257)",4.0,46.0,South Chicago,Southeast,2019-03-28,1553734500000000000,3,Thursday,00:55:00,Night


In [9]:
chicago_crime_df=chicago_crime_df.reset_index(drop=True)
chicago_crime_df

,ID,DateTime,Offense,Place,Arrest,Domestic,Latitude,Longitude,LatLong,Police_Dist,Community_No,Community_Name,Community_Group,Date,DateTime_Numeric,Day_Num,Day_SMTWTFS,Time,Time_of_Day
0,24397,2019-02-05 23:16:00,HOMICIDE,STREET,True,False,41.751181,-87.663461,"(41.751181266, -87.663460692)",6.0,71.0,Auburn Gresham,Far Soutwest,2019-02-05,1549408560000000000,1,Tuesday,23:16:00,Night
1,24426,2019-03-08 01:04:00,HOMICIDE,ALLEY,True,False,41.893646,-87.635629,"(41.893646347, -87.635628657)",18.0,8.0,Near North Side,Central,2019-03-08,1552007040000000000,4,Friday,01:04:00,Night
2,24430,2019-03-13 00:50:00,HOMICIDE,STREET,False,False,41.750322,-87.666211,"(41.750321833, -87.666210811)",6.0,71.0,Auburn Gresham,Far Soutwest,2019-03-13,1552438200000000000,2,Wednesday,00:50:00,Night
3,24437,2019-03-17 13:54:00,HOMICIDE,Other,False,False,41.793773,-87.689421,"(41.793772993, -87.689420934)",9.0,63.0,Gage Park,Southwest,2019-03-17,1552830840000000000,6,Sunday,13:54:00,Midday
4,24451,2019-03-28 00:55:00,HOMICIDE,Other,False,False,41.740262,-87.557553,"(41.740262274, -87.557553257)",4.0,46.0,South Chicago,Southeast,2019-03-28,1553734500000000000,3,Thursday,00:55:00,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,12675976,2022-03-29 20:30:00,ASSAULT,RESIDENCE,False,True,41.922533,-87.707151,"(41.9225328, -87.707150967)",14.0,22.0,Logan Square,North,2022-03-29,1648585800000000000,1,Tuesday,20:30:00,Evening
49996,12675999,2022-02-05 08:35:00,DECEPTIVE PRACTICE,Other,False,False,41.773440,-87.659197,"(41.773440046, -87.659196538)",7.0,67.0,West Englewood,Southwest,2022-02-05,1644050100000000000,5,Saturday,08:35:00,Morning
49997,12676014,2022-04-06 00:00:00,MOTOR VEHICLE THEFT,PARKING LOT,False,False,41.871521,-87.625930,"(41.871520566, -87.625930316)",1.0,32.0,The Loop,Central,2022-04-06,1649203200000000000,2,Wednesday,00:00:00,Night
49998,12676105,2022-04-14 09:26:00,OFFENSE INVOLVING CHILDREN,RESIDENCE,False,False,41.908119,-87.762133,"(41.908118616, -87.762132864)",25.0,25.0,Austin,West,2022-04-14,1649928360000000000,3,Thursday,09:26:00,Morning


In [10]:
# Drop all unnecessary columns for this model

# KEEP COLUMNS
#  Offense
#  Arrest
#  Domestic 
#  DateTime_Numeric
#  Day_Num
#  Time_of_Day
#  Place 
#  Police_Dist
#  Community_Group
#  Latitude
#  Longitude

drop_columns = ['ID',
                'DateTime',
                'Date',
                'Day_SMTWTFS',
                'Time',
                'Day_SMTWTFS',
                'Community_No',
                'Community_Name',
                'LatLong']
                
chicago_crime_df = chicago_crime_df.drop(drop_columns, 1)
chicago_crime_df.head()

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,Offense,Place,Arrest,Domestic,Latitude,Longitude,Police_Dist,Community_Group,DateTime_Numeric,Day_Num,Time_of_Day
0,HOMICIDE,STREET,True,False,41.751181,-87.663461,6.0,Far Soutwest,1549408560000000000,1,Night
1,HOMICIDE,ALLEY,True,False,41.893646,-87.635629,18.0,Central,1552007040000000000,4,Night
2,HOMICIDE,STREET,False,False,41.750322,-87.666211,6.0,Far Soutwest,1552438200000000000,2,Night
3,HOMICIDE,Other,False,False,41.793773,-87.689421,9.0,Southwest,1552830840000000000,6,Midday
4,HOMICIDE,Other,False,False,41.740262,-87.557553,4.0,Southeast,1553734500000000000,3,Night


In [11]:
chicago_crime_df.dtypes

Offense              object
Place                object
Arrest               object
Domestic             object
Latitude            float64
Longitude           float64
Police_Dist         float64
Community_Group      object
DateTime_Numeric      int64
Day_Num               int64
Time_of_Day          object
dtype: object

## Convert columns to a modeling format

### 'Offense' column: One Hot Encoder for string types

In [12]:
# Create the OneHotEncoder instance on the Offense column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(cc_df.Offense.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['Offense'])
encode_df.head()

,Offense_ASSAULT,Offense_BATTERY,Offense_BURGLARY,Offense_CRIMINAL DAMAGE,Offense_DECEPTIVE PRACTICE,Offense_HOMICIDE,Offense_HUMAN TRAFFICKING,Offense_KIDNAPPING,Offense_MOTOR VEHICLE THEFT,Offense_NARCOTICS,Offense_NON-VIOLENT OTHER,Offense_OFFENSE INVOLVING CHILDREN,Offense_SEXUAL CRIME,Offense_SEXUAL VIOLENT_OTHER,Offense_THEFT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# Merge the two DataFrames together and drop the Offense column
chicago_crime_df = chicago_crime_df.merge(encode_df,left_index=True,right_index=True).drop("Offense",1)
chicago_crime_df.head()

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Place,Arrest,Domestic,Latitude,Longitude,Police_Dist,Community_Group,DateTime_Numeric,Day_Num,Time_of_Day,...,Offense_HOMICIDE,Offense_HUMAN TRAFFICKING,Offense_KIDNAPPING,Offense_MOTOR VEHICLE THEFT,Offense_NARCOTICS,Offense_NON-VIOLENT OTHER,Offense_OFFENSE INVOLVING CHILDREN,Offense_SEXUAL CRIME,Offense_SEXUAL VIOLENT_OTHER,Offense_THEFT
0,STREET,True,False,41.751181,-87.663461,6.0,Far Soutwest,1549408560000000000,1,Night,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,ALLEY,True,False,41.893646,-87.635629,18.0,Central,1552007040000000000,4,Night,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,STREET,False,False,41.750322,-87.666211,6.0,Far Soutwest,1552438200000000000,2,Night,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Other,False,False,41.793773,-87.689421,9.0,Southwest,1552830840000000000,6,Midday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Other,False,False,41.740262,-87.557553,4.0,Southeast,1553734500000000000,3,Night,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 'Arrest' & 'Domestic': get_dummies for boolean types

In [14]:
# Binary encoding using Pandas (multiple columns)
chicago_crime_df = pd.get_dummies(chicago_crime_df, columns=["Arrest", "Domestic"])
chicago_crime_df.head()

,Place,Latitude,Longitude,Police_Dist,Community_Group,DateTime_Numeric,Day_Num,Time_of_Day,Offense_ASSAULT,Offense_BATTERY,...,Offense_NARCOTICS,Offense_NON-VIOLENT OTHER,Offense_OFFENSE INVOLVING CHILDREN,Offense_SEXUAL CRIME,Offense_SEXUAL VIOLENT_OTHER,Offense_THEFT,Arrest_False,Arrest_True,Domestic_False,Domestic_True
0,STREET,41.751181,-87.663461,6.0,Far Soutwest,1549408560000000000,1,Night,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,0
1,ALLEY,41.893646,-87.635629,18.0,Central,1552007040000000000,4,Night,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0,1,1,0
2,STREET,41.750322,-87.666211,6.0,Far Soutwest,1552438200000000000,2,Night,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0
3,Other,41.793773,-87.689421,9.0,Southwest,1552830840000000000,6,Midday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0
4,Other,41.740262,-87.557553,4.0,Southeast,1553734500000000000,3,Night,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,1,0


### 'Time_of_Day' column: One Hot Encoder

In [15]:
# Create the OneHotEncoder instance on the Time_of_Day column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(chicago_crime_df.Time_of_Day.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['Time_of_Day'])
encode_df.head()

,Time_of_Day_Afternoon,Time_of_Day_Early Morning,Time_of_Day_Evening,Time_of_Day_Midday,Time_of_Day_Morning,Time_of_Day_Night
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
# Merge the two DataFrames together and drop the Time_of_Day column
chicago_crime_df = chicago_crime_df.merge(encode_df,left_index=True,right_index=True).drop("Time_of_Day",1)
chicago_crime_df.head()

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Place,Latitude,Longitude,Police_Dist,Community_Group,DateTime_Numeric,Day_Num,Offense_ASSAULT,Offense_BATTERY,Offense_BURGLARY,...,Arrest_False,Arrest_True,Domestic_False,Domestic_True,Time_of_Day_Afternoon,Time_of_Day_Early Morning,Time_of_Day_Evening,Time_of_Day_Midday,Time_of_Day_Morning,Time_of_Day_Night
0,STREET,41.751181,-87.663461,6.0,Far Soutwest,1549408560000000000,1,0.0,0.0,0.0,...,0,1,1,0,0.0,0.0,0.0,0.0,0.0,1.0
1,ALLEY,41.893646,-87.635629,18.0,Central,1552007040000000000,4,0.0,0.0,0.0,...,0,1,1,0,0.0,0.0,0.0,0.0,0.0,1.0
2,STREET,41.750322,-87.666211,6.0,Far Soutwest,1552438200000000000,2,1.0,0.0,0.0,...,1,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0
3,Other,41.793773,-87.689421,9.0,Southwest,1552830840000000000,6,0.0,1.0,0.0,...,1,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0
4,Other,41.740262,-87.557553,4.0,Southeast,1553734500000000000,3,0.0,0.0,0.0,...,1,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0


### 'Place' column: One Hot Encoder

In [17]:
# Create the OneHotEncoder instance on the Place column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(chicago_crime_df.Place.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['Place'])
encode_df.head()

,Place_ALLEY,Place_GAS STATION CONVENIENCE,Place_Other,Place_PARKING LOT,Place_PUBLIC TRANSIT,Place_RESIDENCE,Place_RESTAURANT,Place_SIDEWALK,Place_STREET
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Merge the two DataFrames together and drop the Place column
chicago_crime_df = chicago_crime_df.merge(encode_df,left_index=True,right_index=True).drop("Place",1)
chicago_crime_df.head()

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Latitude,Longitude,Police_Dist,Community_Group,DateTime_Numeric,Day_Num,Offense_ASSAULT,Offense_BATTERY,Offense_BURGLARY,Offense_CRIMINAL DAMAGE,...,Time_of_Day_Night,Place_ALLEY,Place_GAS STATION CONVENIENCE,Place_Other,Place_PARKING LOT,Place_PUBLIC TRANSIT,Place_RESIDENCE,Place_RESTAURANT,Place_SIDEWALK,Place_STREET
0,41.751181,-87.663461,6.0,Far Soutwest,1549408560000000000,1,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,41.893646,-87.635629,18.0,Central,1552007040000000000,4,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,41.750322,-87.666211,6.0,Far Soutwest,1552438200000000000,2,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,41.793773,-87.689421,9.0,Southwest,1552830840000000000,6,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,41.740262,-87.557553,4.0,Southeast,1553734500000000000,3,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 'Community_Group' column: : One Hot Encoder

In [19]:
# Create the OneHotEncoder instance on the Community_Group column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(chicago_crime_df.Community_Group.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['Community_Group'])
encode_df.head()

,Community_Group_Central,Community_Group_Far North,Community_Group_Far Soutwest,Community_Group_North,Community_Group_Northwest,Community_Group_South,Community_Group_Southeast,Community_Group_Southwest,Community_Group_West
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [20]:
# Merge the two DataFrames together and drop the Community_Group column
chicago_crime_df = chicago_crime_df.merge(encode_df,left_index=True,right_index=True).drop("Community_Group",1)
chicago_crime_df.head()

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Latitude,Longitude,Police_Dist,DateTime_Numeric,Day_Num,Offense_ASSAULT,Offense_BATTERY,Offense_BURGLARY,Offense_CRIMINAL DAMAGE,Offense_DECEPTIVE PRACTICE,...,Place_STREET,Community_Group_Central,Community_Group_Far North,Community_Group_Far Soutwest,Community_Group_North,Community_Group_Northwest,Community_Group_South,Community_Group_Southeast,Community_Group_Southwest,Community_Group_West
0,41.751181,-87.663461,6.0,1549408560000000000,1,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41.893646,-87.635629,18.0,1552007040000000000,4,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,41.750322,-87.666211,6.0,1552438200000000000,2,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,41.793773,-87.689421,9.0,1552830840000000000,6,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,41.740262,-87.557553,4.0,1553734500000000000,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Build the Model

In [21]:
# Split our preprocessed data into our features and target arrays
y = chicago_crime_df["Arrest_True"].values
X = chicago_crime_df.drop(["Arrest_True","Arrest_False"],1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


### 'DateTime_Numeric', 'Police_Dist', 'Latitude', 'Longitude' columns: fit standard scaler for large ranges of numerical values

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
from keras.models import Sequential
from keras.layers import Dense
number_input_features = len(X_train[0])

def build_model():
    model = Sequential()
    model.add(Dense(16, input_dim=number_input_features, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
keras_model = build_model()
keras_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
782/782 [==============================] - 8s 2ms/step - loss: 0.4836 - accuracy: 0.7855
Epoch 2/100
782/782 [==============================] - 1s 2ms/step - loss: 0.4228 - accuracy: 0.8345
Epoch 3/100
782/782 [==============================] - 1s 2ms/step - loss: 0.4187 - accuracy: 0.8352
Epoch 4/100
782/782 [==============================] - 1s 2ms/step - loss: 0.4156 - accuracy: 0.8360
Epoch 5/100
782/782 [==============================] - 1s 2ms/step - loss: 0.4139 - accuracy: 0.8366
Epoch 6/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4123 - accuracy: 0.8363
Epoch 7/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4109 - accuracy: 0.8363
Epoch 8/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4098 - accuracy: 0.8368
Epoch 9/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4087 - accuracy: 0.8371
Epoch 10/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4082 - accura

782/782 [==============================] - 2s 2ms/step - loss: 0.3880 - accuracy: 0.8414
Epoch 82/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3878 - accuracy: 0.8430
Epoch 83/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3880 - accuracy: 0.8417
Epoch 84/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3879 - accuracy: 0.8424
Epoch 85/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3876 - accuracy: 0.8418
Epoch 86/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3873 - accuracy: 0.8430
Epoch 87/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3871 - accuracy: 0.8437
Epoch 88/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3868 - accuracy: 0.8426
Epoch 89/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3868 - accuracy: 0.8431
Epoch 90/100
782/782 [==============================] - 2s 2ms/step - loss: 0.3863 - accuracy: 

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = keras_model.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 [==============================] - 2s 2ms/step - loss: 0.4394 - accuracy: 0.8238
Loss: 0.43940162658691406, Accuracy: 0.8238000273704529
